In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random as rnd
from matplotlib import colors
import math
import os
import imageio
from collections import defaultdict
from random import randrange, sample

In [ ]:
def initialization(L, Nall, Nblue):
    """
    Function initializes the table with LxL squares where the red and blue agents are placed randomly, what prepares the first
    step of the Shelling's Segregation algorithm
    
    Arguments:
    L -- size of the side of a field square -- integer
    Nall - number of all agents - integer
    Nblue - number of blue agents - integer < Nall, (where number of red agents = Nall-Nblue)
    
    Function returns:
    Nb - List with the placement of blue agents - list
    Nr- List with the placement of red agents - list
    empty_places - list of empty places
    cmap - the map with values that relate to the colors of each cell that will be used in creating images,
    """
    Nred = Nall-Nblue
    N = []
    cmap = list(range(L**2))
    all_places = range(L**2)
    N = rnd.sample(all_places, Nall)  
    Nb = rnd.sample(N,Nblue)
    Nr = list(set(N) - set(Nb))
    empty_places = list(set(all_places) - set(N))
    for i in Nb:
        cmap[i] = 2
    for i in Nr:
        cmap[i] = 1
    for i in empty_places:
        cmap[i] = 0  
    cmap = np.array(cmap).reshape(L,L)
    empty_places.sort()
    return Nb, Nr, empty_places, cmap


def neighbour(L, x, y, m):
    """
    Function returns the cells that are considered as a Moore's neighbourhood of a given
    tree cell (as coords of its row and column), with the given size of the neighborhood, m \in '{8,24,48,80,120}'
    
    Arguments:
    L - size of the side of a forest - integer
    x - row of a cell - integer from range [0,L]
    y - column of a cell - integer from range [0,L]
    m - number of considered neighbours - integer
    
    Function returns the cells that are included in the Moore's neighbourhood 
    
    """
    xx = x
    yy = y
    back = []
    neigh = []
    neighbors = lambda x, y : [[x2, y2] for x2 in range(x-1, x+2)
                                for y2 in range(y-1, y+2)
                                if (x != x2 or y != y2)]
    nei = neighbors(x,y)
    neigh = nei
    nei_2 = []
    if m > 8:
        for l,d in nei:
            nei_2.append(neighbors(l,d))
        neigh = sum(nei_2,[])
    nei_3 = []
    if m>24:
        for l,d in neigh:
            nei_3.append(neighbors(l,d))
        neigh = sum(nei_3,[])
    nei_4 = []      
    if m>48:
        for l,d in neigh:
            nei_4.append(neighbors(l,d)) 
        neigh = sum(nei_4,[])
    nei_5 = []
    if m>80:
        for l,d in neigh:
            nei_5.append(neighbors(l,d)) 
        neigh = sum(nei_5,[])
          
    all_neighbours = []
    for i in neigh:
        if i not in all_neighbours:
            all_neighbours.append(i) 
    if [xx,yy] in all_neighbours:
        all_neighbours.remove([xx,yy])
    for i in all_neighbours:
        if i[0] < 0 :
            i[0] += L+1
        elif i[0] > L:
            i[0] -= L+1
        if i[1] < 0:
            i[1] += L+1
        elif i[1] > L:
            i[1] -= L+1
        back.append((L+1)*i[0]+i[1])
    return back

def read_row_col(cell, L):
    """
    Helper function converting the number of a given cell number into its coordinates.
    
    Arguments:
    cell - number of an agent in the field - integer from range[0,LxL-1]
    L - size of the side of a field square - integer
    
    Function returns the tuple of the coordinates of a given agent
    """
    col = cell%L
    row = math.floor(cell/L)
    return (row,col)

def intersection(lst1, lst2):
    """
    Helper function that finds the intersection of two lists and returns these values
    
    Arguments:
    lst1 - list
    lst2 - list
    """
    lst = [value for value in lst1 if value in lst2]
    return lst

def algorithm_simulation(L, Nb, Nr, empty, cmap, jb, jr, mb = 8, mr = 8, img = True, iterat = 1000):
    """
    Function that simulates the process of Shelling's segregation of agents which are blue or red, taking the arguments which
    were creted by the initialization function and:
    jb, jr - to-stay ratios for blue and red agents - floats
    mb, mr - the size of the considered neighbourhood - int
    img - indicated if the images are to be generated and saved -Boolean
    iterat - number of the iterations of the algorithm
    
    Function returns the final segregation index and number of iterations needed to reach the final state
    """
    for k in range(iterat):
        counter = 0
        if img == True:
            image(cmap,cmap,k)
        ratios = []
        taken = Nb + Nr
        new_Nb = []
        new_Nr = []
        new_taken = taken
        for i in taken:
            row, col = read_row_col(i, L)
            neighbor_list = neighbour(L-1,row,col,mb)
            blues = intersection(neighbor_list, Nb)
            reds = intersection(neighbor_list, Nr)
            if i in Nb:
                if len(reds) + len(blues) != 0:
                    blues_ratio = len(blues)/(len(reds)+len(blues))
                else:
                    blues_ratio = 1
                ratios.append(blues_ratio) 
                if blues_ratio < jb:
                    new_place = rnd.sample(empty,1)[0]
                    row_n, col_n = read_row_col(new_place, L)
                    new_Nb.append(new_place)
                    empty.remove(new_place)
                    empty.append(i)
                    cmap[row,col] = 0
                    cmap[row_n,col_n] = 2
                    counter+=1
                else:
                    new_Nb.append(i)

            elif i in Nr:
                if len(reds) + len(blues) != 0:
                    reds_ratio = len(reds)/(len(reds)+len(blues))
                else:
                    reds_ratio = 1
                ratios.append(reds_ratio)
                if reds_ratio < jr:
                    new_place = rnd.sample(empty,1)[0]
                    row_n, col_n = read_row_col(new_place, L)
                    new_Nr.append(new_place)
                    empty.remove(new_place)
                    empty.append(i)
                    cmap[row,col] = 0
                    cmap[row_n,col_n] = 1
                    counter+=1
                else:
                    new_Nr.append(i)
        if counter == 0:
            segregation_index = sum(ratios)/L**2
            break
        if k == iterat-1:
            segregation_index = sum(ratios)/L**2
        Nr = new_Nr
        Nb = new_Nb
    return  segregation_index, k
    

def image(data, cmap, no):
    """
    Function creates an image of an actual state of a burning forest and saves it in the folder.
    It represents the burning forest with colors as follows:
    green - living tree
    white - an empty square
    red - a burning tree
    black - a burned tree
    
    Aguments:
    data -- actual state of the forest - numpy array
    cmap -- color map of the state of the forest - numpy array
    no - number of the step of the simulation - integer
    p - initial probability (described in initialization) - float from range [0,1]
    
    Function doesn't return anything, just saves images
    """
    cmap = colors.ListedColormap(['white', 'red', 'blue'])
    bounds=[-0.5,0.5,1.5,2.5]
    norm = colors.BoundaryNorm(bounds, cmap.N)
    fig, ax = plt.subplots()
    img = plt.imshow(data, interpolation='nearest', origin='lower',
                        cmap=cmap, norm=norm)
    plt.xlabel("x axis")
    plt.ylabel("y axis")
    plt.title("Schelling’s segregation")

    fig = plt.gcf()
    fig.savefig("Figure" + str(no) + ".png",dpi = 200)
    plt.close()        
    
def create_gif(path, duration):
    """
    Function creates gif from the images that are already in the given directory
    
    Arguments: 
    path - path where the images are being stored - string
    duration - duration in seconds of a each step of the simulation in the gif
    
    Function only creates gif and doesn't return anything.
    """
    images = []
    dirs = os.listdir(path)
    new_images = sorted(dirs, key = len)
    for file in new_images:
        if file.endswith(".png"):
                images.append(imageio.imread(file))
    kargs = { 'duration': duration }
    imageio.mimsave(path+"\\Schelling segregation_2.gif", images,**kargs) 


In [ ]:
init = initialization(100, 8000, 4000)
sse, itr = algorithm_simulation(100, init[0], init[1], init[2], init[3], 0.8, 0.8,mb = 8, mr = 8, img = False, iterat = 50) 
#create_gif("C:\\Users\\user\\Desktop\\Applied mathematics\\Semestr 2\\Agent based modelling\\List2", 0.4)

In [ ]:
r = range(500,8050,50)
iterations = np.zeros(len(r))
for t in range(50):
    for ii in r:
        init = initialization(100, ii, int(ii/2))
        s, k = algorithm_simulation(100, init[0], init[1], init[2], init[3], 0.5, 0.5,mb = 8, mr = 8, img = False) 
        iterations[ii] += k/50

    

In [ ]:
plt.figure()
plt.plot(r,iterations,'-o',markersize=2)
plt.ylabel("Average number of iterations")
plt.xlabel("Number of agents")
plt.title("Number of iterations vs number of agents")
fig = plt.gcf()
fig.savefig("Agents_vs_iterations.png",dpi = 500)
plt.show()

In [ ]:
rang = [x / 100 for x in range(10, 95, 5)]
segregation = np.zeros(len(rang))
for u in range(50):
    for i in rang:
        init = initialization(100, 8000, 4000)
        seg, kk = algorithm_simulation(100, init[0], init[1], init[2], init[3], i, i,mb = 8, mr = 8, img = False) 
        segregation[i] += seg/50

In [ ]:
plt.figure()
plt.plot(rang,segregation,'-o',markersize=2)
plt.ylabel("Segregation index")
plt.xlabel("to-stay ratio")
plt.title("Segregation index vs to-stay ratio")
fig = plt.gcf()
fig.savefig("Segregation_vs_ratio.png",dpi = 500)
plt.show()

In [ ]:
nei = [1,2,3,4,5]
segregation_2 = np.zeros(len(nei))
for o in range(50):
    for m in nei:
        init = initialization(100, 8000, 4000)
        segr, k = algorithm_simulation(100, init[0], init[1], init[2], init[3], 0.5, 0.5,mb = (2*m+1)**2-1, mr = (2*m+1)**2-1, img = False)
        segregation_2[m] += segr/50

In [ ]:
plt.figure()
plt.plot(nei,segregation_2,'-o',markersize=2)
plt.ylabel("Segregation index")
plt.xlabel("Neighbourhood size")
plt.title("Segregation index vs size of neighbourhood")
fig = plt.gcf()
fig.savefig("Segregation_vs_neighbour.png",dpi = 500)
plt.show()